In [2]:
import numpy as np 
import pandas as pd 
import os
import tensorflow.keras as keras
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D, MaxPooling2D, Conv2D, Flatten, Dropout
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.applications import VGG16

In [2]:
import glob as gb 
import cv2

In [3]:
IMG_SIZE = 224

TRAINING_DIR = '../data/familia_simpsons_train/'

training_datagen = ImageDataGenerator(rescale=1./255, featurewise_center=False,
 samplewise_center=False, 
 featurewise_std_normalization=False, 
 samplewise_std_normalization=False, 
 rotation_range=0, 
 width_shift_range=0.1, 
 height_shift_range=0.1, 
 horizontal_flip=True, 
 vertical_flip=False)

train_generator = training_datagen.flow_from_directory(TRAINING_DIR,target_size=(IMG_SIZE,IMG_SIZE), class_mode='categorical', batch_size=64, shuffle=True)

Found 7274 images belonging to 6 classes.


In [4]:
class_map = train_generator.class_indices
inv_class_map = {v: k for k, v in class_map.items()}
inv_class_map

{0: 'abraham_grampa_simpson',
 1: 'bart_simpson',
 2: 'homer_simpson',
 3: 'lisa_simpson',
 4: 'maggie_simpson',
 5: 'marge_simpson'}

In [5]:
#wp = '../weights.best.hdf5'
vgg16_base = VGG16(include_top=False,
                   input_tensor=None, input_shape=(IMG_SIZE, IMG_SIZE, 3))

In [6]:
vgg16_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [7]:
print('Adding new layers...')
output = vgg16_base.get_layer(index = -1).output
output = Flatten()(output)
output = Dense(6, activation='softmax')(output)
print('New layers added!')

Adding new layers...
New layers added!


In [8]:
vgg16_model = Model(vgg16_base.input, output)
for layer in vgg16_model.layers[:-7]:
    layer.trainable = False
vgg16_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [9]:
vgg16_model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics =['accuracy'])

In [10]:
history = vgg16_model.fit_generator(train_generator,
                              epochs=15,
                              verbose=1
                              )

Epoch 1/15
114/114 [==============================] - 1833s 16s/step - loss: 1.7618 - accuracy: 0.2801
Epoch 2/15
114/114 [==============================] - 1767s 15s/step - loss: 1.6293 - accuracy: 0.3217
Epoch 3/15
114/114 [==============================] - 1766s 15s/step - loss: 1.1633 - accuracy: 0.5549
Epoch 4/15
114/114 [==============================] - 1766s 15s/step - loss: 0.6289 - accuracy: 0.7978
Epoch 5/15
114/114 [==============================] - 1766s 15s/step - loss: 0.4332 - accuracy: 0.8617
Epoch 6/15
114/114 [==============================] - 1766s 15s/step - loss: 0.2991 - accuracy: 0.9033
Epoch 7/15
114/114 [==============================] - 1762s 15s/step - loss: 0.2692 - accuracy: 0.9227
Epoch 8/15
114/114 [==============================] - 1767s 15s/step - loss: 0.1795 - accuracy: 0.9445
Epoch 9/15
114/114 [==============================] - 1768s 16s/step - loss: 0.1577 - accuracy: 0.9519
Epoch 10/15
114/114 [==============================] - 1762s 15s/step - l

In [3]:
fig = plt.figure(figsize=(10,5))

plt.subplot(221)
plt.plot(history.history['accuracy'],'bo--', label = "acc")
plt.plot(history.history['loss'],'ro--', label = "loss")
plt.title("train_acc vs loss")
plt.ylabel("accuracy")
plt.xlabel("epochs")
plt.legend()

plt.legend()
plt.show()

NameError: name 'history' is not defined

In [17]:
#vgg16_model.save('VGG16_15_epochs_model.h5')

In [12]:
testpath = '../data/familia_simpsons_test/'
acum = 0
for files in  os.listdir(testpath):
    acum += 1
print(f'For testing data , found {acum} files')

For testing data , found 248 files


In [13]:
s = 224

In [14]:
X_test = []
files = gb.glob(str(testpath + '/*.jpg'))
for file in files: 
    image = cv2.imread(file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_array = cv2.resize(image , (s,s))
    X_test.append(list(image_array))

In [15]:
X_test = np.array(X_test, dtype = 'float32')
print(f'X_test shape : {X_test.shape}')
X_test = X_test/255.0

X_test shape : (248, 224, 224, 3)


In [16]:
predictions = vgg16_model.predict(X_test)
predictions_1 = np.argmax(predictions, axis=-1)
predictions_1

array([0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0,
       0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 2,
       5, 5, 5, 5, 5, 5], dtype=int64)